In [1]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [27]:
# Path ke direktori dataset
dataset_dir = 'D:/@KULIAH SMTR 6/BANGKIT/CAPSTONE/dataset/dataset/'

In [61]:
# Membaca dataset dan melakukan ekstraksi fitur menggunakan FLBP
def extract_features(dataset_dir):
    images = []
    labels = []

    for filename in os.listdir(dataset_dir):
        if filename.endswith('.bmp'):
            image_path = os.path.join(dataset_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (100, 100))  # Mengubah ukuran gambar menjadi 100x100 pixel
            
            # Ekstraksi fitur menggunakan FLBP
            radius = 3
            n_points = 4 * radius
            lbp = local_binary_pattern(image, n_points, radius, method='uniform')
            feature = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))[0]
            
            images.append(feature)
            
            # Mendapatkan angka kolestrol dari nama file
            cholesterol = filename.split('-')[1]
            if cholesterol.isdigit():
                labels.append(int(cholesterol))
            else:
                print("Invalid filename format:", filename)

    return np.array(images), np.array(labels)

In [53]:
# Memanggil fungsi untuk melakukan ekstraksi fitur dari dataset menggunakan FLBP
images, labels = extract_features(dataset_dir)

In [54]:
images

array([[ 1874,  1883,  1557, ...,  1866,  2394, 11312],
       [ 1874,  1883,  1557, ...,  1866,  2394, 11312],
       [ 1472,  1672,  1508, ...,  1695,  1987,  9814],
       ...,
       [ 1651,  1812,  1170, ...,  1785,  2259, 10882],
       [ 2151,  2134,  1252, ...,  2214,  2752, 12702],
       [ 2151,  2134,  1252, ...,  2214,  2752, 12702]], dtype=int64)

In [63]:
# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [64]:
# Membangun model regresi linear
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [57]:
# Lakukan prediksi pada data uji
y_pred = model.predict(X_test)



# Konversi prediksi menjadi kelas menggunakan threshold tertentu (jika diperlukan)
# Misalnya, jika tingkat kolestrol >= 200 dianggap tinggi, maka:
threshold = 200
y_pred_classes = (y_pred >= threshold).astype(int)
y_test_classes = (y_test >= threshold).astype(int)

# Evaluasi model
confusion_mat = confusion_matrix(y_test_classes, y_pred_classes)
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)


In [58]:
# Lakukan prediksi pada data uji
y_pred = model.predict(X_test)

# Ini untuk 3 Class
threshold_low = 200
threshold_high = 240

# Mengkonversi prediksi menjadi kelas berdasarkan threshold
y_pred_classes = np.zeros_like(y_pred)  # Inisialisasi dengan kelas rendah
y_pred_classes[y_pred >= threshold_high] = 2  # Kelas tinggi
y_pred_classes[(y_pred >= threshold_low) & (y_pred < threshold_high)] = 1  # Kelas beresiko

y_test_classes = np.zeros_like(y_test)
y_test_classes[y_test >= threshold_high] = 2
y_test_classes[(y_test >= threshold_low) & (y_test < threshold_high)] = 1

# Evaluasi model
confusion_mat = confusion_matrix(y_test_classes, y_pred_classes)
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

In [59]:
print("Confusion Matrix:")
print(confusion_mat)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


Confusion Matrix:
[[1 0 0]
 [3 7 0]
 [0 3 1]]
Accuracy: 0.6
Precision: 0.75
Recall: 0.6
F1-Score: 0.6


In [65]:
# Evaluasi model
score = model.score(X_test, y_test)
print("Akurasi model:", score)

Akurasi model: 0.4338736823396536


# Try to predict

In [24]:
# Path gambar yang ingin diperiksa
image_path = r'D:\@KULIAH SMTR 6\BANGKIT\CAPSTONE\dataset\dataset\kolesterol-247-14-kanan.bmp'

# Membaca gambar menggunakan cv2.imread()
image = cv2.imread(image_path)

# Memeriksa apakah gambar berhasil dibaca
if image is not None:
    print("Gambar berhasil dibaca.")
    cv2.imshow("Gambar", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Gagal membaca gambar.")

Gambar berhasil dibaca.


In [25]:
from skimage.feature import local_binary_pattern

In [26]:
# Coba Predict data
# Memuat gambar
image_path = r'D:\@KULIAH SMTR 6\BANGKIT\CAPSTONE\dataset\dataset\kolesterol-247-14-kanan.bmp'
image = cv2.imread(image_path)

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))  # Mengubah ukuran gambar menjadi 100x100 pixel
            
# Ekstraksi fitur menggunakan FLBP
radius = 3
n_points = 4 * radius
lbp = local_binary_pattern(image, n_points, radius, method='uniform')
feature = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))[0]

# Mengubah dimensi fitur menjadi bentuk yang dapat diterima oleh model (sesuaikan dengan bentuk fitur yang digunakan saat melatih model)
feature = np.reshape(feature, (1, -1))

# Melakukan prediksi menggunakan model
prediction = model.predict(feature)

# Cetak hasil prediksi
print("Prediksi Kolesterol:", prediction)

Prediksi Kolesterol: [207.13502075]


In [58]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [77]:
# Save Model menggunakan joblib
import joblib

# Simpan model ke dalam file .h5 menggunakan joblib
joblib.dump(model, 'regression_model.h5')

['regression_model.h5']

In [64]:
# Muat kembali model dari file .h5
model = joblib.load('regression_model.h5')

# OTAK ATIK MODEL

In [3]:
# Membaca dataset dan melakukan ekstraksi fitur menggunakan FLBP
def extract_features(dataset_dir):
    images = []
    labels = []

    for filename in os.listdir(dataset_nya):
        if filename.endswith('.bmp'):
            image_path = os.path.join(dataset_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (48, 48))  # Mengubah ukuran gambar menjadi 100x100 pixel
            
            # Ekstraksi fitur menggunakan FLBP
            radius = 3
            n_points = 4 * radius
            lbp = local_binary_pattern(image, n_points, radius, method='uniform')
            feature = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))[0]
            
            images.append(feature)
            
            # Mendapatkan angka kolestrol dari nama file
            cholesterol = filename.split('-')[1]
            if cholesterol.isdigit():
                labels.append(int(cholesterol))
            else:
                print("Invalid filename format:", filename)

    return np.array(images), np.array(labels)

# Try to Augmentasi Data

In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(horizontal_flip=True,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2)
dataset_nya = datagen_train.flow_from_directory('D:/@KULIAH SMTR 6/BANGKIT/CAPSTONE/dataset/dataset/',
                                                    target_size=(100,100),
                                                    color_mode="grayscale",
                                                    shuffle=True)

Found 0 images belonging to 0 classes.
